In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# Define the directory path
output_directory = '/content/data'

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# --- PARTE 1: LÍNEA BASE REALISTA (WBS 100 ACTIVIDADES) ---
fases = ['Ingeniería de Detalle', 'Procura de Equipos', 'Obras Civiles', 'Montaje Electromecánico', 'Pruebas y Puesta en Servicio']
actividades_nombres = [
    "Diseño de Malla a Tierra", "Cálculo de Flujo de Carga", "Planos de Canalizaciones",
    "Compra de Transformador T-01", "Adquisición de Celdas 13.8kV", "Logística de Despacho",
    "Excavación de Fosa de Aceite", "Cimentación de Pórticos", "Construcción de Caseta de Control",
    "Montaje de Seccionadores", "Tendido de Cable de Potencia", "Conexión de Tableros de Protección",
    "Pruebas de Inyección Primaria", "Parametrización de Relés", "Protocolos de Energización"
]

data_wbs = []
for i in range(100):
    fase = fases[i % len(fases)]
    nombre = f"{actividades_nombres[i % len(actividades_nombres)]} - Fase {i//20 + 1}"
    bac = (1000000 * 0.20 / 20) if i < 20 else (1000000 * 0.80 / 80) # Distribución de carga
    data_wbs.append([f"{fases.index(fase)+1}.{i:02d}", nombre, fase, round(bac, 2)])

df_wbs = pd.DataFrame(data_wbs, columns=['WBS', 'Actividad', 'Fase', 'BAC'])
df_wbs.to_csv(os.path.join(output_directory, 'linea_base_subestacion.csv'), index=False)

# --- PARTE 2: DATOS DE CAMPO (CPI=0.82, SPI=0.81) ---
# Simulación de 2500 registros para robustez analítica
n_registros = 2500
spi, cpi = 0.81, 0.82
ev_total_req = (1000000 * 0.85) * spi # Asumiendo 85% transcurrido
ac_total_req = ev_total_req / cpi

data_campo = []
for i in range(n_registros):
    ref = df_wbs.sample(n=1).iloc[0]
    ev_parcial = (ev_total_req / n_registros) * np.random.uniform(0.9, 1.1)
    ac_parcial = ev_parcial / cpi * np.random.uniform(0.95, 1.05)
    data_campo.append([i+1, ref['WBS'], ref['Actividad'], round(ev_parcial, 2), round(ac_parcial, 2), '2026-02-15'])

df_campo = pd.DataFrame(data_campo, columns=['ID', 'WBS', 'Actividad', 'EV', 'AC', 'Fecha'])
df_campo.to_csv(os.path.join(output_directory, 'avance_campo_subestacion.csv'), index=False)